In [3]:
#Stella's Notes: This is the .py file Stella is currently using to write the NLP code for the project.
#Need to do:
#Incorporate age 
#How to save cleaned/stemmed/tokenized lyrics as df? How to save the dicts/inverted indexes etc as well?
#How to prioritize certain words above others. Should I even do this? E.g if someone enter "happy Monday", should I prioritize happy or Monday?
#How to semantically relate words? E.g cold with frozen, etc
#Handle misspelled words

#Create a datasets for: cleaned lyrics, tokenized lyrics, lyric polarized score, other essential stuff.

import json
import os
from flask import Flask, render_template, request
from flask_cors import CORS
from helpers.MySQLDatabaseHandler import MySQLDatabaseHandler
import pandas as pd


spotify_df = pd.read_csv("mini_spotify_track_db.csv")
lyric_df = pd.read_csv("mini_spotify_db.csv")

In [4]:
spotify_df.head()

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [5]:
lyric_df.head()

,artist,song,link,text,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
lyric_df['text']

0      Look at her face, it's a wonderful face  \r\nA...
1      Take it easy with me, please  \r\nTouch me gen...
2      I'll never know why I had to go  \r\nWhy I had...
3      Making somebody happy is a question of give an...
4      Making somebody happy is a question of give an...
                             ...                        
530      \r\n  \r\n  \r\nHey Ah Na Na  \r\nInnocence ...
531    Yeah! Yeah!  \r\n  \r\nYou left the lights on ...
532    When I waken, and I'm achin', time for sleepin...
533    What the hell am I?  \r\nThousand eyes, a fly ...
534    Yeah, I want to travel south this year  \r\nAa...
Name: text, Length: 535, dtype: object

In [7]:
from typing import List, Tuple, Dict
from collections.abc import Callable
import numpy as np
import re

In [8]:
def tokenize(text: str) -> List[str]:
    """Returns a list of words that make up the text.
    
    Note: for simplicity, lowercase everything. Do not remove duplicate words.
    Requirement: Use Regex to satisfy this function
    
    Parameters
    ----------
    text : str
        The input string to be tokenized.

    Returns
    -------
    List[str]
        A list of strings representing the words in the text.
    """
    outcome = re.findall(r"[a-zA-Z]+", text.lower())
    return outcome

In [9]:
list_of_lyrics = lyric_df['text'].tolist()

In [10]:
dict_of_lyrics = {i: row for i, row in enumerate(lyric_df.to_dict(orient='records'))}
#A dictionary of dictionaries. 
#Key: Row number. Value: Dict of {artist:..., link:...., song:..., text:...}

In [11]:
def tokenize_lyrics(dict_of_lyrics, tokenize_method: Callable[[str], List[str]]):
    tokenized_lyrics = {}
    nrows = len(lyric_df)
    for i in range(nrows):
        lyrics = dict_of_lyrics[i]['text']
        tokenized_lyrics[i] = tokenize_method(lyrics)
    return tokenized_lyrics

In [12]:
tokenized_lyrics = tokenize_lyrics(dict_of_lyrics, tokenize)
#A dict with the row numbers of lyric_df as keys and the lsit of tokenized lyrics as value

In [13]:

def build_word_song_count(tokenize_method: Callable[[str], List[str]],
    tokenized_lyrics: Dict[int, List[str]]):
    """Returns a dictionary with the row numbers of songs each distinct word appears in as VALUEs and distinct words as KEYS

    Parameters
    ----------
    tokenize_method : Callable[[str], List[str]]
        A method to tokenize a string into a list of strings representing words.
    tokenized_lyrics: 
        A dict with the row numbers of lyric_df as keys and the lsit of tokenized lyrics as value
    Returns
    -------
    Dict[str, int]
        A dictionary of words mapped to the number of songs they appear in.
    """
    song_count = {}
    
    for key in tokenized_lyrics:
        unique_tokens = set(token.casefold() for token in tokenized_lyrics[key])

        for token in unique_tokens:
            if token not in song_count:
                song_count[token] = set()
            song_count[token].add(key)

    return song_count

In [14]:
song_count = build_word_song_count(tokenize, tokenized_lyrics)
# A dict with KEYS as unique words and VALUES as a list of all the songs(their row numbers) the word appears in

In [15]:

import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
 
nltk.download('stopwords')
print(stopwords.words('english'))

/Applications/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/juanruzhang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
def remove_stop_words(list_of_stop_words,tokenized_lyrics):
    
    """Returns a dictionary with KEYS as row values and VALUES as lyrics without stop words
    Parameters
    ----------
    list_of_stop_words : A list of stop words(E.g and, so, the....)
    tokenized_lyrics: 
        A dict with the row numbers of lyric_df as KEYS and the list of tokenized lyrics as VALUE
    Returns
    -------
    Dict[str, int]
        A dictionary of row numbers mapped to the cleaned tokenized lyrics.
    """
    
    clean_stop_words = set()
    for word in list_of_stop_words:
        clean_stop_words.update(nltk.word_tokenize(word.lower()))  
    
    stop_word_removed_lyrics = {}
    
    for i, lyrics in tokenized_lyrics.items():
        cleaned_lyrics = [word for word in lyrics if word.lower() not in clean_stop_words]
        stop_word_removed_lyrics[i] = cleaned_lyrics
                
    return stop_word_removed_lyrics

In [17]:
cleaned_tokenized_lyrics = remove_stop_words(stopwords.words('english'),tokenized_lyrics)
#A dictionary of row numbers mapped to the cleaned tokenized lyrics.

In [18]:
def remove_stop_words_input(tokenize,list_of_stop_words,input_words):
    
    """Returns a dictionary with KEYS as row values and VALUES as lyrics without stop words
    Parameters
    ----------
    list_of_stop_words : A list of stop words(E.g and, so, the....)
    input_words: A string(The input)
    Returns
    -------
    A LIST of tokenized words with stop words removed
    """
    list_tokens = tokenize(input_words)
    cleaned_words = [word for word in list_tokens if word not in list_of_stop_words]
    
    return cleaned_words

In [19]:
clean_song_count = build_word_song_count(tokenize, cleaned_tokenized_lyrics)
# A dict with KEYS as unique NON STOP words and VALUES as a list of all the songs(their row numbers) the word appears in

In [20]:

#Long runtime
def create_j_sim_mat(input_dict) -> np.ndarray:
    """Create Jaccard similarity matrix for songs.
    Create Jaccard similarity matrix, a np.ndarray of size (`len(lyric_df)`, `len(lyric_df)`),
    computing the character similarity, where the entry (i, j) indicating the Jaccard similarity
    between the songs `i` and `j`.

    Parameters
    ----------
    input_dict: input dictionary of KEYS as song row numbers and VALUES as their lyrics
    Returns
    -------
    np.ndarray
        The Jaccard similarity matrix of (`number of rows`, `number of rows`), with the entry
        (i, j) indicating the Jaccard similarity between the songs `i` and `j`.
    """
    return_matrix = np.zeros((len(lyric_df),len(lyric_df)))

    for song_index in input_dict:
      for other_song_index in input_dict:
        song_words = set(input_dict[song_index])
        other_song_words = set(input_dict[other_song_index])
        union = song_words | other_song_words
        intersection = song_words & other_song_words
        value = (len(intersection)-1)/(len(union)-1) 
        return_matrix[song_index][other_song_index] = value
    
    return return_matrix

In [21]:

def build_inverted_index(songs: dict) -> dict:
    """Builds an inverted index from the song lyrics.

    Arguments
    =========

    songs: dict
        A dictionary where keys are song numbers and values are lists of tokens (words from the lyrics).

    Returns
    =======

    inverted_index: dict
        An inverted index mapping each term to a sorted list of tuples (song_number, term_frequency):
        inverted_index[term] = [(s1, tf1), (s2, tf2), ...]

    Example
    =======

    >> songs = {
    ...    1: ['love', 'me', 'do', 'love', 'love'],
    ...    2: ['hello', 'goodbye', 'hello']
    ... }

    >> idx = build_inverted_index(songs)

    >> idx['love']
    [(1, 3),(2,2)....]

    >> idx['hello']
    [(2, 2)]
    """
    inverted_index = {}

    for song_num, tokens in songs.items():
        token_counts = {}
        
        for token in tokens:
            token_counts[token] = token_counts.get(token, 0) + 1
        
        for token, count in token_counts.items():
            if token not in inverted_index:
                inverted_index[token] = []
            inverted_index[token].append((song_num, count))

    return inverted_index


In [22]:
import math
def compute_idf(inv_idx, n_docs, min_df=0, max_df_ratio=1):
    """Compute term IDF values from the inverted index.
    Words that are too frequent or too infrequent get pruned.

    Hint: Make sure to use log base 2.

    inv_idx: an inverted index as above

    n_docs: int,
        The number of songs.

    min_df: int,
        Minimum number of documents a term must occur in.
        Less frequent words get ignored.
        Documents that appear min_df number of times should be included.

    max_df_ratio: float,
        Maximum ratio of documents a term can occur in.
        More frequent words get ignored.

    Returns
    =======

    idf: dict
        For each term, the dict contains the idf value.

    """

    # TODO-5.1
    return_dict = {}
    
    for word, word_list in inv_idx.items():
        word_count = len(word_list)
        
        if word_count >= min_df and word_count / n_docs <= max_df_ratio:
            idf = math.log2(n_docs/(1+word_count))
            return_dict[word] = idf
                
    return return_dict

In [23]:
inverted_index = build_inverted_index(cleaned_tokenized_lyrics)

In [24]:
tdidf = compute_idf(inverted_index, len(lyric_df), min_df=0, max_df_ratio=1)

In [25]:
def freq_dis(list_of_words):
    """How many times each word appears within a given text. 
    
    Parameters
    ----------
    list_of_words: a list of tokenized words
    """
    fd = nltk.FreqDist(list_of_words)
    return fd

In [26]:
text = nltk.Text(cleaned_tokenized_lyrics[0])
text.concordance("mine", lines=5)

Displaying 2 of 2 matches:
nd girl makes feel fine ever believe mine kind girl without blue ever leaves g
nd girl makes feel fine ever believe mine kind girl without blue ever leaves


In [27]:
#Bigrams: Frequent two-word combinations
#Trigrams: Frequent three-word combinations
#Quadgrams: Frequent four-word combinations
finder = nltk.collocations.TrigramCollocationFinder.from_words(text)
result = finder.ngram_fd.most_common(2)


In [28]:
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/juanruzhang/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [29]:
sia.polarity_scores("Wow, NLTK is not not not terrible!")
# compound ranges from -1 (most negative) to +1 (most positive).

{'neg': 0.0, 'neu': 0.457, 'pos': 0.543, 'compound': 0.7134}

In [30]:
def classify_word_sentiments(list_of_words):
    """_summary_

    Args:
        list_of_words (_type_): list of words

    Returns:
        a dictionary with KEYS as positive, negative, neutral and VALUES of sets of words that belong to each category
    """
    positive = []
    negative = []
    neutral = []

    for word in list_of_words:
        score = sia.polarity_scores(word)["compound"]
        if score > 0:
            positive.append(word)
        elif score < 0:
            negative.append(word)
        else:
            neutral.append(word)
    
    return {"positive":positive, "negative":negative, "neutral":neutral}


In [31]:
from itertools import chain
from nltk.corpus import wordnet
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

In [32]:
def get_synonyms_of_word(word):
    """_summary_

    Args:
        word (_type_): word

    Returns:
        _type_: a set of all the synonyms of the word
    """
    synonyms = []

    for syn in wordnet.synsets(word):
        for i in syn.lemmas():
            synonyms.append(i.name().replace("_", " ").lower())

    return set(synonyms)

In [33]:
for key in cleaned_tokenized_lyrics:
    if "love you" in " ".join(cleaned_tokenized_lyrics[key]):
        #print(key)
        continue

In [34]:
# Stemming words
ps = PorterStemmer()
ps.stem("words")

'word'

ACTUAL IMPLEMENTATION START HERE
Step 1: Get inputs
2: Use genre to search for songs
3: Find exact matching words 

In [35]:
from collections import Counter
# Find exact matching words 
# "genre_input" is the user inputted genre
user_genre_input = "gorge"

#Tokenize and clean(Remove stop words) user input
#clean_genre_input : {0:[Cleaned user input]}
clean_genre_input = remove_stop_words(stopwords.words('english'),{0:tokenize(user_genre_input)})

#A dict with KEYS as unique NON STOP words !!from the user input!! and VALUES as a list of all the songs(their row numbers) the word appears in
possible_songs_dict = {}

for word in clean_genre_input[0]:
    if clean_song_count.get(word) is not None:
        #Clean_song_count: A dict with KEYS as unique NON STOP words and VALUES as a list of all the songs(their row numbers) the word appears in
        possible_songs_dict[word] = clean_song_count[word]
print(possible_songs_dict.keys())
    
#Stem the words that can't be found in song lyrics and add them to stemmed_user_input
stemmed_user_input = []
#Stem the words that can't be found in song lyrics and add them to stemmed_user_input, preserve words that are already found in song lyrics
stemmed_user_input_preserve_original = []
for word in clean_genre_input[0]:
    if possible_songs_dict.get(word) is None:
        stemmed_user_input_word = ps.stem(word)
        stemmed_user_input.append(stemmed_user_input_word)
        stemmed_user_input_preserve_original.append(stemmed_user_input_word)
    else:
        stemmed_user_input_preserve_original.append(word)

#Update the list of possible songs by searching for stemmed words in the songs
for word in stemmed_user_input:
    if clean_song_count.get(word) is not None:
        possible_songs_dict[word] = clean_song_count[word]
print(possible_songs_dict.keys())

#Get the songs that has the most number of relevant words from user input
most_common_songs = []
if possible_songs_dict:
    # Count occurrences of each song index
    song_counts = Counter(song for song_list in possible_songs_dict.values() for song in song_list)
    # Find the maximum count
    max_number = max(song_counts.values(), default=0)
    # Return the song indices that appear the most
    most_common_songs = [song for song, count in song_counts.items() if count == max_number]

#For words in the user input that don't exist in any songs, find the synonyms
# A dictionary with KEYS as user input words that don't exist in any songs, VALUES as a list of their synonyms
word_synonym_dict = {}
for word in stemmed_user_input_preserve_original:
    if possible_songs_dict.get(word) is None:
        set_of_synonyms = get_synonyms_of_word(word)
        word_synonym_dict[word] = list(set_of_synonyms)
print(word_synonym_dict)
##NEED TO FIX: Not sure how to incorporate synonyms


dict_keys([])
dict_keys([])
{'gorg': []}


In [36]:
print(most_common_songs)

[]


In [37]:
#Get the total number(including repeats) of inputted words in song
if len(most_common_songs) > 40:
    most_common_songs = most_common_songs[:40]  

sum_of_words_dict = {}

for word in clean_genre_input[0]:
    if word not in inverted_index:
        continue  

    inv_index_list = inverted_index[word]
    for song_idx, num_words in inv_index_list:
        if song_idx not in most_common_songs:
            continue

        sum_of_words_dict[song_idx] = sum_of_words_dict.get(song_idx, 0) + num_words

top_30_songs = sorted(sum_of_words_dict, key=sum_of_words_dict.get, reverse=True)[:30]


In [38]:
print(top_30_songs)

[]


In [39]:
#Find songs without(or with very little) antonyms
def get_antonyms(word):
    antonyms = []
    for syn in wordnet.synsets(word):
        for i in syn.lemmas():
            if i.antonyms():
                antonyms.append(i.name().replace("_", " ").lower())
    return set(antonyms)

In [40]:
# Step 1: Get all antonyms for user input words
word_antonyms = {word: get_antonyms(word) for word in clean_genre_input[0]}

# for the top_30_songs, find the songs with these antonyms
song_antonym_counts = Counter()

for word, antonyms in word_antonyms.items():
    for antonym in antonyms:
        if antonym in clean_song_count:
            for song in clean_song_count[antonym]:
                if song in top_30_songs: 
                    song_antonym_counts[song] += 1 

# Step 3: Add songs with no antonyms with values of 0
for song in top_30_songs:
    if song not in song_antonym_counts:
        song_antonym_counts[song] = 0

# Step 4: Sort songs by least distinct antonyms
filtered_songs = sorted(song_antonym_counts.keys(), key=lambda song: song_antonym_counts[song])

filtered_songs = filtered_songs[:10]



In [41]:
print(filtered_songs)

[]


In [42]:

#Sort the input_song_list based on the polarity score(pos, neg, neutral, compound(Compound being -1 to 1, with -1 being neg and 1 being pos))
def sort_polarity_scores(input_song_list, cleaned_tokenized_lyrics, polarity_type="compound"):
    sia = SentimentIntensityAnalyzer() 
    dict_of_polarity_scores = {}

    for song_row_number in input_song_list:
        if song_row_number in cleaned_tokenized_lyrics:
            dict_of_polarity_scores[song_row_number] = sia.polarity_scores(" ".join(cleaned_tokenized_lyrics[song_row_number]))

    # Sort songs by the specified polarity score in descending order
    sorted_songs = sorted(dict_of_polarity_scores, key=lambda x: dict_of_polarity_scores[x][polarity_type], reverse=True)

    return sorted_songs

In [43]:
#Takes too long to run
result = sort_polarity_scores(filtered_songs, cleaned_tokenized_lyrics, "pos")

In [44]:
print(result)

[]


In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer

song_lyrics = cleaned_tokenized_lyrics.values()

joined_song_lyrics = []
for lyrics in song_lyrics:
    joined_song_lyrics.append(" ".join(lyrics))

vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(joined_song_lyrics)

svd = TruncatedSVD(n_components=min(tfidf_matrix.shape))
svd.fit(tfidf_matrix)
explained_variance = np.cumsum(svd.explained_variance_ratio_)

optimal_num_topics = np.argmax(explained_variance >= 0.90) + 1


In [55]:
words = vectorizer.get_feature_names_out()

In [ ]:
#Which index corresponds to what word
print(words[3])

aaar


In [63]:
svd = TruncatedSVD(n_components=optimal_num_topics)
svd.fit(tfidf_matrix)

for topic_idx, topic in enumerate(svd.components_):
    top_words_idx = topic.argsort()[::-1][:10]
    top_words_for_topic = [words[i] for i in top_words_idx]
    print(f"Topic {topic_idx + 1}: {' '.join(top_words_for_topic)}")

Topic 1: love know want chorus like let time away say got
Topic 2: love heart forever chorus true hold want away christmas close
Topic 3: love yeah la girl got goin hey little baby said
Topic 4: la girl watch mama oh better betty life said world
Topic 5: want know make baby la let got better gonna close
Topic 6: yeah la walk away chorus turn come let fly live
Topic 7: let girl dance come love yeah baby shine music dancing
Topic 8: world turn make friendly game card ends depends hey think
Topic 9: girl hey baby walk na away chorus kind sa remember
Topic 10: na hey sa ang chorus christmas gonna ko ng happy
Topic 11: man want na sa woman ang yeah close live world
Topic 12: man na sa ang let got gonna walk gotta la
Topic 13: walk christmas away gotta somebody dance want turn say round
Topic 14: little baby christmas chorus friendly let world ends card stop
Topic 15: hey come wanna gonna girl chorus tell believe darling baby
Topic 16: girl chorus want queen dancing gonna like dance river ye

In [65]:
topics_for_songs = svd.transform(tfidf_matrix)
dom_topics= np.argsort(-topics_for_songs, axis=1)[:, :10]

# Print dom topics for first 5 songs
for i in range(5):
    print(f"Song {i}'s Top 10 topics: {dom_topics[i] + 1}")

Song 0's Top 10 topics: [  9   1  40   7  16   4  33  46 128  27]
Song 1's Top 10 topics: [286 242 306 265 195   1 218 169 267 282]
Song 2's Top 10 topics: [ 1 40 26 36 13 68 23 45 35 56]
Song 3's Top 10 topics: [ 59  54  64  55 177 151 163 143  61 135]
Song 4's Top 10 topics: [59 54 64 55 61 40 35 78 39 77]


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def find_song_matches_with_svd(user_input, tfidf_vectorizer, svd, topics_for_songs, song_titles, song_artists):
    user_tfidf = tfidf_vectorizer.transform([user_input])
    user_topics = svd.transform(user_tfidf)

    similarities = cosine_similarity(user_topics, topics_for_songs)

    top_indices = np.argsort(-similarities[0])[:5]
    
    # Return matches
    return [(song_titles[idx], song_artists[idx], similarities[0, idx]) for idx in top_indices]

# Example Usage
matches = find_song_matches_with_svd('walk with me my love here in the forests', vectorizer, svd, topics_for_songs, list(lyric_df['song']), list(lyric_df['artist']))

for rank, (title, artist, score) in enumerate(matches, start=1):
    print(f"Match {rank}: {title} {artist}(Similarity: {score})")


Match 1: Lonesome Valley Alabama(Similarity: 0.7736665196391537)
Match 2: Walk On Down Aerosmith(Similarity: 0.7105783358635683)
Match 3: Don't Walk Away Air Supply(Similarity: 0.5881483503502678)
Match 4: If You Love Me Air Supply(Similarity: 0.3276703069488694)
Match 5: Then Again Alabama(Similarity: 0.31217595573472434)
